### Helpers and Imports

In [34]:
import random
import sys
from typing import Tuple
sys.setrecursionlimit(1000000)

def is_prime(n, k):
    # Miller-Rabin primality test
    
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


def generate_big_prime(size):
    k = 40
    p = random.randrange(2 ** (size - 1), 2 ** size - 1)
    if p % 2 == 0:
        p += 1
    while not is_prime(p, k):
        p += 2
    return p


def egcd(a: int, b: int) -> Tuple[int, int, int]:
    # https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Recursive_algorithm_2
    """return (g, x, y) such that a*x + b*y = g = gcd(a, b)"""
    if a == 0:
        return (b, 0, 1)
    else:
        b_div_a, b_mod_a = divmod(b, a)
        g, x, y = egcd(b_mod_a, a)
        return (g, y - b_div_a * x, x)


def modinv(a: int, b: int) -> int:
    # https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Modular_inverse
    """return x such that (x * a) % b == 1"""
    g, x, _ = egcd(a, b)
    if g != 1:
        raise Exception('gcd(a, b) != 1')
    return x % b

# Pallier Cryptosystem

https://en.wikipedia.org/wiki/Paillier_cryptosystem

https://doi.org/10.1007%2F3-540-48910-X_16

In [38]:
def gen(n):
    # 1. Choose two large prime numbers of equal length
    length = 30
    p = generate_big_prime(length)
    q = generate_big_prime(length)

    # 2a. Compute n = pq and lambda = lcm(p-1, q-1)    OR
    # 2b. Compute n = pq and lambda = phi(n) = (p - 1) * (q - 1)
    n = p * q
    lambda_ = (p - 1) * (q - 1)
    
    # 3a. Select random integer g from Z_{n^2}^*    OR
    # 3b. Compute g = n + 1
    g = n + 1
    
    # 4a. Ensure n divides the order of g and calculate mu = (L(g^lambda mod n^2))^{-1} mod n    OR
    # 4b. Calculate mu = phi(n)^{-1} mod n = lambda^{-1} mod n
    mu = modinv(lambda_, n)
    
    # Set the private key and secret key
    pk = (n, g)
    sk = (lambda_, mu)
    return pk

def enc(pk, m):
    pass

def dec(sk, pk, m):
    pass

# PLONK

https://eprint.iacr.org/2019/953.pdf

In [2]:
def gen(n):
    pass

def enc(pk, m):
    pass

def dec(sk, pk, m):
    pass

In [9]:
is_prime(1201, 40)

True

In [37]:
gen(11)

(1015240156817636400, 971412167008371604)
1


(1015240158833040263, 1015240158833040264)

In [29]:
(35 * 15) % 5

0